Здесь приведена версия алгоритма Берри из работы Закари Шерра 2013. Оптимально работает для порядков 1-8. Возможно и выше, устойчивость не проверена. Планируется добавить автоматическую остановку по достижению периода (когда встречается 2a0), чтобы снизить вычислительную нагрузку.

In [ ]:
R.<a,b> = PolynomialRing(QQ)
K       = R.fraction_field()
S.<x>   = PolynomialRing(K)

r2 = (8*a - 2) / b^2
r1 = 32*a       / b^3
r0 = (16*a^2 + 24*a + 1) / b^4

d  = x^4 + r2*x^2 + r1*x + r0
B  = x^2 + r2/2


L  = S(0)      # L_0 = 0
M  = S(1)      # M_0 = 1

ai  = []      
ri  = []      
Li  = [L]
Mi  = [M]

max_iterations = 7

for i in range(max_iterations):
    q, r = (L + B).quo_rem(M)
    ai.append(q)
    ri.append(r)
    
    L_next = B - r
    if i == 0:
        M_next = d - L_next^2
    else:
        M_next = Mi[i-1] + q * (r - ri[i-1])
    
    Li.append(L_next)
    Mi.append(M_next)
    L, M = L_next, M_next


print("\nНеполные частные a_i(x)")
for k, a_k in enumerate(ai):
    sym_expr = SR(a_k)
    simp = sym_expr.simplify()
    print(f"  a_{k}(x) = {simp}")


Данный код пока не обобщён на другие случаи, планируется организовать это как фреймворк, который может работать со значениями для любых порядков.

In [6]:
var('x b c')
assume(c != 0, b != 0)  

d = (x^4
     + (8*c - 2)/b^2         * x^2
     + 32*c/b^3              * x
     + (16*c^2 + 24*c + 1)/b^4)


a0 = x^2               + (4*c - 1)/b^2
a1 = (b^3/(16*c))*x    -  b^2/(16*c)
a2 = (4/b)*x           -  4/b^2
alist = [a0, a1, a2]     


p_prev2, p_prev1 = 0, 1      #  P_{-2}, P_{-1}
q_prev2, q_prev1 = 1, 0      #  Q_{-2}, Q_{-1}

for a_k in alist:
    p_i = (a_k*p_prev1 + p_prev2).simplify_full()
    q_i = (a_k*q_prev1 + q_prev2).simplify_full()
    p_prev2, p_prev1 = p_prev1, p_i
    q_prev2, q_prev1 = q_prev1, q_i

P, Q = p_prev1, q_prev1 

delta   = (P^2 - d*Q^2).simplify_full()
QP_coef = (-1/delta).simplify_full() 

print(f"q_period = {len(alist)};  period = {2*len(alist)};  "
      f"QP_coef = {QP_coef}")

print(f"deg(P)= {P.degree(x)}, P: {P}")
print(f"deg(Q)= {Q.degree(x)}, Q: {Q}")

print("Check Pell: ", delta)


q_period = 3;  period = 6;  QP_coef = 1/64*b^4/c
deg(P)= 4, P: 1/4*(b^4*x^4 - 2*b^3*x^3 + 8*b^2*c*x^2 + 16*c^2 + 2*(4*b*c + b)*x - 16*c - 1)/(b^2*c)
deg(Q)= 2, Q: 1/4*(b^2*x^2 - 2*b*x + 4*c + 1)/c
Check Pell:  -64*c/b^4


Простенькая фильтрация

In [3]:
R.<x> = PolynomialRing(QQ)
solutions = []

for alpha in range(0, 5):
    for beta in range(0, 7):
        if (alpha + beta <= 6
            and 3*alpha + 2*beta >= 12
            and 4*alpha + 2*beta >= 12
            and alpha <= 4
            and 10*alpha + 6*beta <= 44):

            a = 2^(34 - 8*alpha - 6*beta)
            b = 2^(14 - 3*alpha - 2*beta)
            c = 2^(8  - 2*alpha -   beta)

            P = (b^4 * x^4
                 - 2*b^3 * x^3
                 + 8*a*b^2 * x^2
                 + 2*(4*a + 1)*b * x
                 + (16*a^2 - 16*a - 1)
                ) / (4*a*b^2)

            Q = (b^2 * x^2
                 - 2*b * x
                 + (4*a + 1)
                ) / (4*a)

            f = c * P
            g = c * Q

            if f in R and g in R:
                d = (f^2 + 1) / g^2
                if d in R:
                    if d.gcd(d.derivative()) == 1:
                        solutions.append((alpha, beta, a, b, c, d, f, g))
                    else:
                        solutions.append((alpha, beta, a, b, c, d, f, g, "d не свободен от квадратов"))

if not solutions:
    print("Ни одна пара (α,β) не дала нетривиального решения.")
else:
    for sol in solutions:
        alpha, beta, a, b, c, d, f, g = sol[:8]
        print(f"α={alpha}, β={beta}")
        print(f"a={a}, b={b}, c={c}")
        print(f"d(x)={d}")
        print(f"f(x)={f}")
        print(f"g(x)={g}")
        if len(sol)>8:
            print("  —", sol[8])
        print("-----------")


α=0, β=6
a=1/4, b=4, c=4
d(x)=x^4 + 1/8*x + 1/32
f(x)=64*x^4 - 32*x^3 + 8*x^2 + 4*x - 1
g(x)=64*x^2 - 32*x + 8
-----------
α=1, β=5
a=1/16, b=2, c=2
d(x)=x^4 - 3/8*x^2 + 1/4*x + 41/256
f(x)=32*x^4 - 32*x^3 + 4*x^2 + 10*x - 31/8
g(x)=32*x^2 - 32*x + 10
-----------
α=2, β=3
a=1, b=4, c=2
d(x)=x^4 + 3/8*x^2 + 1/2*x + 41/256
f(x)=8*x^4 - 4*x^3 + 4*x^2 + 5/4*x - 1/32
g(x)=8*x^2 - 4*x + 5/2
-----------
α=2, β=4
a=1/64, b=1, c=1
d(x)=x^4 - 15/8*x^2 + 1/2*x + 353/256
f(x)=16*x^4 - 32*x^3 + 2*x^2 + 34*x - 319/16
g(x)=16*x^2 - 32*x + 17
-----------
α=3, β=2
a=1/4, b=2, c=1
d(x)=x^4 + x + 1/2
f(x)=4*x^4 - 4*x^3 + 2*x^2 + 2*x - 1
g(x)=4*x^2 - 4*x + 2
-----------
α=4, β=0
a=4, b=4, c=1
d(x)=x^4 + 15/8*x^2 + 2*x + 353/256
f(x)=x^4 - 1/2*x^3 + 2*x^2 + 17/32*x + 191/256
g(x)=x^2 - 1/2*x + 17/16
-----------
